In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorly
!pip install pmdarima
%cd /content/drive/My Drive/Colab Notebooks/Group30_Final_Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 163kB 8.0MB/s 
  Created wheel for tensorly: filename=tensorly-0.4.5-cp36-none-any.whl size=100156 sha256=5600098bc25238a3e38eca8fe40ffd40d87e2be904eb188ae7227a5f71992761
  Stored in directory: /root/.cache/pip/wheels/44/ae/02/8d00229a4fd0af192b48d24da903f9975c7ac10e706685fc39
Successfully built tensorly
     |████████████████████████████████| 1.5MB 2.8MB/s 
/content/

In [0]:
%cd /content/drive/My Drive/Colab Notebooks/Group30_Final_Project

/content/drive/.shortcut-targets-by-id/1onvLc9hmQDF89OPg7Mt7Gu3hNH54oJiI/Group30_Final_Project


In [0]:
from input.datasets_from_paper.transform_ele_1096 import dataset_ele_1096

In [0]:
import numpy as np

from BHT_ARIMA import BHTARIMA
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_BHTARIMA(data):


      p = 3 # p-order
      d = 2 # d-order
      q = 1 # q-order
      taus = [321,500] # MDT-rank
      Rs = [5,5] # tucker decomposition ranks
      k =  15 # iterations
      tol = 0.001 # stop criterion
      Us_mode = 3 # orthogonality mode
      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      for i in range(T_test):

        ts = data[..., i:T-T_test+i].copy()
        n_round += 1
        model = BHTARIMA(ts, p, d, q, taus, Rs, k, tol, verbose=0, Us_mode=Us_mode)
        result, _ = model.run()
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [0]:
BHTARIMA_dataset_ele_1096 = run_BHTARIMA(dataset_ele_1096)
BHTARIMA_dataset_ele_1096

{'acc': 0.9951875732241426, 'nrmse': 0.014214127811633337}

In [6]:
import numpy as np
import pmdarima as pm
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_ARIMA(data):

      testsize = 0.1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        y = data[..., i:T-T_test+i].copy()
        n_round += 1

        for j in range(y.shape[0]):
          
          model = pm.AutoARIMA(suppress_warnings=True,trace=False,error_action="ignore")

          result = model.fit_predict(y[j], n_periods=1)
          result_full[j, i] = result[..., -1]


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
ARIMA_dataset_ele_1096 = run_ARIMA(dataset_ele_1096)
ARIMA_dataset_ele_1096

{'acc': 0.9954093873758737, 'nrmse': 0.01269416194425352}

In [0]:
import numpy as np
import pandas as pd

from fbprophet import Prophet
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_Prophet(data):


      testsize = 0.1
      start = '2017-3'
      freq = 'D'


      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      n_round = 0

      time_st = pd.period_range(start=start, periods=T, freq=freq).to_timestamp()


      for i in range(T_test):

        y = data[..., i:T-T_test+i+1].copy()
        n_round += 1
        ds = time_st[i:T-T_test+i+1].copy()
        


        for j in range(y.shape[0]):


          DF_ts = pd.DataFrame()
          DF_ts['ds'] = ds.copy()
          DF_ts['y'] = y[j].copy()



          model = Prophet()
          model.fit(DF_ts[i:T-T_test+i])

          result = model.predict(DF_ts[['ds']].iloc[[-1]])['yhat'][0]
          result_full[j, i] = result


      true_value = data[..., -T_test:]


      stat = {}
      stat['acc'] = get_acc(result_full, true_value)
      stat['nrmse'] = nrmse(result_full, true_value)


      return(stat)
 

In [0]:
Prophet_dataset_ele_1096 = run_Prophet(dataset_ele_1096)
Prophet_dataset_ele_1096

{'acc': 0.9617017917815542, 'nrmse': 0.10011510915795638}

In [0]:
import numpy as np

from statsmodels.tsa.api import VAR
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_VAR(data):
  
      testsize = 0.1
      p = 1

      T = data.shape[-1]
      T_test = int((T * testsize) // 1)
      result_full = np.zeros([data.shape[0], T_test])

      total_time = 0
      n_round = 0

      for i in range(T_test):

        ts = (data[..., i:T-T_test+i].copy())
        n_round += 1
        model = VAR(ts)
        result = model.fit(p).forecast(ts, 1)
        result_full[..., i] = result[..., -1]

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [0]:
VAR_dataset_ele_1096= run_VAR(dataset_ele_1096)
VAR_dataset_ele_1096

{'acc': 0.20667130476909498, 'nrmse': 4.424782367272446}

In [0]:
import numpy as np

import xgboost as xgb
from BHT_ARIMA.util.utility import get_acc, nrmse

def run_XGB(data, test_size):

      T_test = test_size

      result_full = np.zeros([data.shape[0], T_test])
      parameters = {'max_depth':(3,4,5,6,7,8,9,10), 
              'subsample':[0.5,0.6, 0.7, 0.8, 0.9],
               'lambda' :[0, 0.25, 0.5, 0.75, 1]}
      clf = GridSearchCV(XGBRegressor(objective ='reg:squarederror'), parameters, verbose = False, cv = 3)

      X = data[..., :-T_test-1]
      y = data[..., -T_test-1]
      clf.fit(X, y)
      for i in range(T_test):
        X_test = data[..., i+1:-T_test+i]
        result = clf.predict(X_test)
        result_full[..., i] = result

      label = data[..., -T_test:]

      stat = {}
      stat['acc'] = get_acc(result_full, label)
      stat['nrmse'] = nrmse(result_full, label)

      return(stat)

In [13]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore")

XGB_dataset_ele_1096= run_XGB(dataset_ele_1096, test_size=dataset_ele_1096.shape[1] // 10)
XGB_dataset_ele_1096

{'acc': 0.9340026745489344, 'nrmse': 1.7875561979996497}